In [ ]:
import numpy as np
import extended_cir
import loan_simulation
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def set_seed_extended_cir(seed=100):
    extended_cir.np.random.seed(seed)


def set_seed_loan_simulation(seed=123):
    loan_simulation.np.random.seed(seed)


def generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params):
    ir_model = extended_cir.InterestRateCIRPP(ir_termS, ir_params['k'][0], ir_params['theta'][0], ir_params['sigma'][0],
                                              ir_params['x0'][0])
    cr_model = extended_cir.CreditRiskCIRPP(ir_termS, cr_termS, cr_params['k'][0], cr_params['theta'][0],
                                            cr_params['sigma'][0], cr_params['x0'][0], cr_params['recovery_rate'][0],
                                            cr_params['premium_frequency'][0])
    ssrd = extended_cir.SSRDSimulation(ir_model, cr_model, ssrd_params['rho'], ssrd_params['simulations'],
                                       ssrd_params['time_step'], ssrd_params['T'])
    return ssrd


def simulate_funding_cost(ssrd, liquidity_risk_percent, risk_margin_percent):
    set_seed_extended_cir()
    rt, ct, ft = ssrd.simulateMC()
    ft += liquidity_risk_percent
    ft += risk_margin_percent
    return ft, rt, ct


def plot_funding_cost(ft, rt, ct):
    plt.figure(figsize=(12, 8))
    plt.plot(rt.T)
    plt.title("Interest Rate(Risk Free)")
    plt.figure(figsize=(12, 8))
    plt.plot(ct.T)
    plt.title('Hazard Rate')
    plt.figure(figsize=(12, 8))
    plt.plot(ft.T)
    plt.title('Funding Cost');


def simulate_loan_tranches():
    set_seed_loan_simulation()
    loan_sim = loan_simulation.LoanBookSimulation(60, 10 * 1e7, 0.5)
    loan_sim.simulate_loan_book()
    loan_book = loan_sim.loan_book_segmentation();
    return loan_sim.split_loan_into_tranches(loan_book)


def yearly_to_monthly_rate(rate):
    return (1 + rate) ** (1 / 12) - 1


def survivorship(loan_rate, interest_rate, cric_p=0.33):
    # return loan_rate <= interest_rate
    h = 100
    p = 1 / (1 + np.exp(-h * (interest_rate - loan_rate)))
    return p > cric_p


def calculate_clv(loan_tranche, simulations, max_loan_term, ft, rt):
    loan_amount = sum(loan_tranche['loan_amount'])
    total_loans = len(loan_tranche)
    fixed_rate = sum(loan_tranche['fixed_rate']) / total_loans
    loan_term = int(sum(loan_tranche['loan_term']) / total_loans)
    monthly_rate = fixed_rate / 12
    clvs = []
    loan_life = []
    emi = loan_amount * (monthly_rate * (1 + monthly_rate) ** loan_term) / ((1 + monthly_rate) ** loan_term - 1)
    for sim_num in range(simulations):
        clv = np.zeros(max_loan_term)
        last_month = loan_term
        outstanding_principal = loan_amount
        for month in range(1, loan_term + 1):
            monthly_interest = outstanding_principal * monthly_rate
            principal_component = emi - monthly_interest
            outstanding_principal -= principal_component
            funding_cost = 1 + yearly_to_monthly_rate(ft[sim_num, month])
            cf = monthly_interest / (funding_cost ** month)
            if not survivorship(fixed_rate, rt[sim_num, month]):
                last_month = month
                clv[month - 1:max_loan_term] = clv[month - 2] + 3 * cf
                break
            clv[month - 1:max_loan_term] = clv[month - 2] + cf
        clvs.append(clv / total_loans)
        loan_life.append(last_month)

    return np.array(clvs), np.array(loan_life)


def find_clv(tranches, ft, rt, max_loan_term):
    tranche_clvs = []
    labels = []
    for key, loan_tranche in tranches.items():
        clvs, loan_life = calculate_clv(loan_tranche, ssrd_params['simulations'], max_loan_term, ft, rt)
        labels.append(key)
        tranche_clvs.append(clvs.mean(axis=0))
    tranche_clvs = np.array(tranche_clvs)
    labels = np.array(labels)
    clv_table = pd.DataFrame({'label': labels, 'clv': tranche_clvs.T[:][-1]})
    return labels, tranche_clvs, clv_table


def plot_clv(labels, clvs, max_loan_term, top_tranches_to_disply, image_name):
    # return
    high_clv_indices = sorted_indexes_desc(clvs[:, max_loan_term - 1])
    plt.figure(figsize=(12, 8))
    plt.plot(clvs[high_clv_indices[:top_tranches_to_disply]].T,
             label=labels[high_clv_indices[:top_tranches_to_disply]]);
    plt.title('Evolution of CLV for each Tranche over 60 months');
    leg_title = "Legend(Amount, Term, Fixed Rate)"
    leg = plt.legend(title=leg_title)  #, loc='upper left', bbox_to_anchor=(1, 1))
    plt.setp(leg.get_title(), weight='bold')
    plt.savefig(f'{image_name}.png')
    plt.show()

def sorted_indexes_desc(clvs):
    high_clv_indices = np.argsort(clvs)
    return high_clv_indices[::-1]

def display_clv(clvs_df):
    high_clv_indices = sorted_indexes_desc(np.array(clvs_df['clv']))
    print(clvs_df.iloc[high_clv_indices].to_string(index=False))

def write_clv_to_excel(clvs, sheet_names):
    with pd.ExcelWriter('clv_result.xlsx', engine='xlsxwriter') as writer:
        for clv_table, sheet_name in zip(clvs, sheet_names):
            clv_table.to_excel(writer, sheet_name=sheet_name, index=False)

### Simulate Funding Cost Using SSRD Model

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
liquidity_risk_bps = np.linspace(0.0025, 0.01, 61)
risk_margin_bps = 0.005
max_loan_tenure = 60
top_tranche_clv_to_display = 10
clv_tables = []
clv_sheet_names = []
ssrd_params = {'simulations': 1000, 'time_step': 1 / 12, 'T': 5, 'rho': 0.3}
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
plot_funding_cost(funding_cost, risk_free_rate, credit_premia)

### Find CLV under normal condition

In [ ]:
loan_tranches = simulate_loan_tranches()
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('no_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'without_stress');
display_clv(clv_for_excel)

### Under Interest rate 1.1 Vol Stress

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_params['sigma'] *= 1.1
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_interest_inc_vol_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_interest_increase_vol_stress');
display_clv(clv_for_excel)

### Under 0.9 interest rate vol stress

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_params['sigma'] *= 0.9
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_interest_dec_vol_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_interest_decrease_vol_stress');
display_clv(clv_for_excel)

### Under credit risk 1.1 vol stress

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
cr_params['sigma'] *= 1.1
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_credit_inc_vol_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_credit_increase_vol_stress');
display_clv(clv_for_excel)

### Under credit risk 0.9 vol stress

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
cr_params['sigma'] *= 0.9
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_credit_dec_vol_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_credit_increase_dec_stress');
display_clv(clv_for_excel)

### Under interest rate parallel shift stress +10 bps

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] += 0.001
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_inc_10bps_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_inc_10bps_stress');
display_clv(clv_for_excel)

### Under interest rate parallel shift stress -10 bps

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] -= 0.001
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_dec_10bps_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_dec_10bps_stress');
display_clv(clv_for_excel)

### Under credit spread parallel shift stress +10 bps

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
cr_termS['spread'] += 0.001
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_cre_term_inc_10bps_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_cre_term_inc_10bps_stress');
display_clv(clv_for_excel)

### Under credit spread parallel shift stress -10 bps

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
cr_termS['spread'] -= 0.001
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_cre_term_dec_10bps_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_cre_term_dec_10bps_stress');
display_clv(clv_for_excel)

### With Normal yield curve(downward curve)

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
max_index = np.argmax(ir_termS['rate'])
ir_termS.loc[max_index:, 'rate'] = ir_termS['rate'][max_index]
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_interest_normalised')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_interest_normalised');
display_clv(clv_for_excel)

### Under less stress on initial curve and more on the end

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] += 0.001
ir_termS.loc[len(ir_termS)/2:, 'rate'] += 0.001
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_back_more_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_backend_more_stress');
display_clv(clv_for_excel)

### Under combined stress in vol and yield curve interest rate(increase)

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] += 0.001
ir_params['sigma'] *= 1.1
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_vol_inc_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_vol_increase_stress');
display_clv(clv_for_excel)

### Under combined stress in vol and yield curve interest rate(decrease)

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] -= 0.001
ir_params['sigma'] *= 0.9
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_vol_dec_stress')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_vol_increase_stress');
display_clv(clv_for_excel)

### Under combined stress in vol(increase) and yield curve(decrease) interest rate

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] -= 0.001
ir_params['sigma'] *= 1.1
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_dec_vol_inc')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_dec_vol_inc');
display_clv(clv_for_excel)

### Under combined stress in vol(decrease) and yield curve(increase) interest rate

In [ ]:
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
ir_termS['rate'] += 0.001
ir_params['sigma'] *= 0.9
ssrd = generate_ssrd_model(ir_termS, cr_termS, ir_params, cr_params, ssrd_params)
funding_cost, risk_free_rate, credit_premia = simulate_funding_cost(ssrd, liquidity_risk_bps, risk_margin_bps)
clv_labels, trancheClvs, clv_for_excel = find_clv(loan_tranches, funding_cost, risk_free_rate, max_loan_tenure)
clv_tables.append(clv_for_excel)
clv_sheet_names.append('with_int_term_inc_vol_dec')
plot_clv(clv_labels, trancheClvs, max_loan_tenure, top_tranche_clv_to_display, 'with_int_term_inc_vol_dec');
display_clv(clv_for_excel)

In [ ]:
write_clv_to_excel(clv_tables, clv_sheet_names)